<a href="https://colab.research.google.com/github/MehrdadJalali-AI/GraphMOF-AI/blob/main/GraphMOF_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')
# Change working path
os.chdir('/content/drive/MyDrive/Research/MOF/GAN-NodeGeneration/')
!pip install rdkit

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 8.2 MB/s eta 0:00:00


In [4]:
!pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.47-cp311-cp311-linux_x86_64.whl size=24577465 sha256=f29d911124d0b65f3d24d66c353314930893045135b6ee07f9b21281900a29a8
  Stored in directory: /root/.cache/pip/wheels/25/9a/a3/fb1ac6339caa859c8bb18d685736168b0b51d851af13d81d52
Successfully built owlready2


In [9]:
import pandas as pd
from owlready2 import *

# Load the MOF dataset
file_path = "MOF.csv"
mof_data = pd.read_csv(file_path)

# Create a new ontology
onto = get_ontology("http://example.org/MOF_Ontology.owl")

with onto:
    # Main Classes
    class MOF(Thing):
        """Metal-Organic Framework"""

    class MetalComponent(Thing):
        """Metal-based part of MOF"""

    class MetalIon(MetalComponent):
        """Primary metal ion present in the MOF"""

    class MetalCluster(MetalComponent):
        """Secondary building unit (SBU) formed by linked metal ions"""

    class OrganicLinker(Thing):
        """Organic part of MOF responsible for porosity and functionalization"""

    class Ligand(OrganicLinker):
        """Specific chemical species forming organic linkers"""

    class StructuralProperty(Thing):
        """Geometrical and topological properties of the MOF"""

    class FunctionalProperty(Thing):
        """Functional behavior of the MOF"""

    # Define Data Properties Separately (without multiple inheritance)
    class CoordinationNumber(DataProperty):
        """Number of ligands/atoms coordinated to the central metal"""
        range = [int]
        is_a = [MetalComponent >> int]  # Assign domain

    class VoidFraction(DataProperty):
        """Fraction of the MOF that is empty space"""
        range = [float]
        is_a = [MOF >> float]

    class SurfaceArea(DataProperty):
        """Accessible surface area for gas adsorption"""
        range = [float]
        is_a = [MOF >> float]

    class PoreLimitingDiameter(DataProperty):
        """Smallest pore size allowing guest molecules"""
        range = [float]
        is_a = [MOF >> float]

    class LargestCavityDiameter(DataProperty):
        """Largest cavity size in the MOF"""
        range = [float]
        is_a = [MOF >> float]

    class LargestFreePathDiameter(DataProperty):
        """Longest free path for guest diffusion"""
        range = [float]
        is_a = [MOF >> float]

    # Define Object Properties for Relationships
    class has_metal_component(ObjectProperty):
        domain = [MOF]
        range = [MetalComponent]

    class has_organic_linker(ObjectProperty):
        domain = [MOF]
        range = [OrganicLinker]

    class has_structural_property(ObjectProperty):
        domain = [MOF]
        range = [StructuralProperty]

    class has_functional_property(ObjectProperty):
        domain = [MOF]
        range = [FunctionalProperty]

# Populate Ontology from CSV
def populate_ontology_from_csv():
    for index, row in mof_data.iterrows():
        mof_instance = MOF(row['Refcode'])

        # Add Metal Ion Component
        if not pd.isna(row['metals']):
            metal_ion_instance = MetalIon(row['metals'])
            mof_instance.has_metal_component.append(metal_ion_instance)

        # Add Data Properties
        if not pd.isna(row['void_fraction']):
            mof_instance.void_fraction = float(row['void_fraction'])

        if not pd.isna(row['pld (A)']):
            mof_instance.pore_limiting_diameter = float(row['pld (A)'])

        if not pd.isna(row['asa (A^2)']):
            mof_instance.surface_area = float(row['asa (A^2)'])

populate_ontology_from_csv()

# Save the ontology
onto.save("MOF_Ontology.owl")
print("Ontology saved as MOF_Ontology.owl")


Ontology saved as MOF_Ontology.owl


In [12]:
from py2neo import Graph, Node, Relationship
import pandas as pd

# Connect to Neo4j (Modify the connection details accordingly)
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

# Load the MOF dataset
file_path = "MOF.csv"
mof_data = pd.read_csv(file_path)

graph.delete_all()  # Clear existing data

# Create MOF nodes and relationships
for index, row in mof_data.iterrows():
    mof_node = Node("MOF", name=row['Refcode'])
    graph.create(mof_node)

    # Metal Component
    if not pd.isna(row['metals']):
        metal_node = Node("Metal", name=row['metals'])
        graph.merge(metal_node, "Metal", "name")
        graph.create(Relationship(mof_node, "HAS_METAL", metal_node))

    # Void Fraction
    if not pd.isna(row['void_fraction']):
        void_node = Node("Property", name="Void Fraction", value=float(row['void_fraction']))
        graph.create(void_node)
        graph.create(Relationship(mof_node, "HAS_PROPERTY", void_node))

    # Pore Limiting Diameter
    if not pd.isna(row['pld (A)']):
        pld_node = Node("Property", name="PLD", value=float(row['pld (A)']))
        graph.create(pld_node)
        graph.create(Relationship(mof_node, "HAS_PROPERTY", pld_node))

    # Surface Area
    if not pd.isna(row['asa (A^2)']):
        sa_node = Node("Property", name="Surface Area", value=float(row['asa (A^2)']))
        graph.create(sa_node)
        graph.create(Relationship(mof_node, "HAS_PROPERTY", sa_node))

print("MOF data successfully stored in Neo4j.")


ConnectionUnavailable: Cannot open connection to ConnectionProfile('bolt://localhost:7687')

In [11]:
!pip install py2neo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 3.3 MB/s eta 0:00:00
